# Investigación técnica melee.gg (torneo 339227)

Objetivos:
1. Verificar si el HTML inicial contiene datos de rondas/matches o si se cargan por JS.
2. Identificar endpoints internos/API para rounds/matches.
3. Definir formato de campos clave para matriz de emparejamientos.

Este notebook ejecuta inspección HTTP y mapea evidencia reproducible.

In [1]:
import json, re, ssl, urllib.request, urllib.parse
from datetime import datetime, timezone

BASE = 'https://melee.gg'
TOURNAMENT_ID = '339227'
URL = f'{BASE}/Tournament/View/{TOURNAMENT_ID}'

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
}

ctx = ssl.create_default_context()

req = urllib.request.Request(URL, headers=headers)
with urllib.request.urlopen(req, context=ctx, timeout=30) as resp:
    html = resp.read().decode('utf-8', errors='replace')
    status = resp.status
    final_url = resp.geturl()
    response_headers = dict(resp.getheaders())

print('status:', status)
print('final_url:', final_url)
print('html_len:', len(html))
print('content_type:', response_headers.get('Content-Type'))
print('set_cookie_present:', any(k.lower() == 'set-cookie' for k,_ in resp.getheaders()))

status: 200
final_url: https://melee.gg/Tournament/View/339227
html_len: 124605
content_type: text/html; charset=utf-8
set_cookie_present: False


In [2]:
print('--- HTML snippet ---')
print('\n'.join(html.splitlines()[:40]))

checks = {
    'window.__INITIAL_STATE__': 'window.__INITIAL_STATE__' in html,
    'window.__STATE__': 'window.__STATE__' in html,
    'application/json script tags': bool(re.search(r'<script[^>]+type=["\']application/json["\']', html, re.I)),
    'tournament id present as text': TOURNAMENT_ID in html,
    'match keywords in HTML': bool(re.search(r'"match|round|bracket|entrant|participant"', html, re.I)),
}
print('\n--- Embedded data checks ---')
for k,v in checks.items():
    print(f'{k}: {v}')

--- HTML snippet ---
<!DOCTYPE html>
<html>
<head>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"/>

<title>Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am | Melee</title>
<impact-site-verification value="4c243f18-8267-493f-a951-c8659ec151bc"></impact-site-verification>
<meta content="@MeleePlatform" property="twitter:site"></meta>
<meta content="0.3.63.85" name="version"></meta>
<meta content="Magic: the Gathering,Magic Arena,MTGA" name="keywords"></meta>
<meta content="https://cdn.melee.gg/userphotos/brandimages/2df961f6-de44-4764-bfdc-b375be26a74a.jpg" property="og:image"></meta>
<meta content="A brand image for the tournament." property="twitter:image:alt"></meta>
<meta content="https://melee.gg/Tournament/View/339227" property="og:url"></meta>
<meta content="website" property="og:type"></meta>
<meta content="summary" property="twitte

In [3]:
script_srcs = re.findall(r'<script[^>]+src=["\']([^"\']+)["\']', html, re.I)
script_urls = []
for src in script_srcs:
    if src.startswith('http://') or src.startswith('https://'):
        script_urls.append(src)
    elif src.startswith('//'):
        script_urls.append('https:' + src)
    else:
        script_urls.append(urllib.parse.urljoin(BASE, src))

print('scripts_found:', len(script_urls))
for u in script_urls[:30]:
    print(u)

scripts_found: 29
https://cdn.melee.gg/scripts/site.min.js?v=0.3.63.85
https://cdn.melee.gg/scripts/font-awesome.min.js?v=0.3.63.85
https://melee.gg/Scripts/flattsware.core.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.filters.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.enums.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.standings.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.datatables.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.datetime.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.encode.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.regex.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.notify.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.spinner.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.forms.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.decklist.min.js?v=0.3.63.85&lg=en
https://melee.gg/Scripts/flattsware.idle.min

In [4]:
# Descargar JS para buscar rutas internas
js_texts = {}
for u in script_urls[:20]:
    try:
        req = urllib.request.Request(u, headers={'User-Agent': headers['User-Agent'], 'Accept': '*/*', 'Referer': URL})
        with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
            ctype = (r.headers.get('Content-Type') or '').lower()
            if 'javascript' in ctype or u.endswith('.js'):
                js_texts[u] = r.read().decode('utf-8', errors='replace')
    except Exception as e:
        pass

print('js_downloaded:', len(js_texts))

pattern = re.compile(r'(/[A-Za-z0-9_\-]+(?:/[A-Za-z0-9_\-]+){1,6}(?:\?[A-Za-z0-9_\-=&%]+)?)')
candidates = set()
for u,txt in js_texts.items():
    for m in pattern.findall(txt):
        if any(key in m.lower() for key in ['api','tournament','round','match','bracket','standing','standings','pairing','event']):
            candidates.add(m)

candidates = sorted(candidates)
print('candidate_paths:', len(candidates))
for p in candidates[:200]:
    print(p)

js_downloaded: 20
candidate_paths: 12
/Standing/GetStandingsConfig
/Tournament/GetPlayerDetails
/Tournament/GetTeamDetails
/Tournament/GetTournamentDetails
/Tournament/PlayerCard
/Tournament/View
/compat/matcher
/edit_session/bracket_match
/lib/event
/lib/event_emitter
/mouse/mouse_event
/selection/eventRelay


In [ ]:
# Probar candidatos (GET) y detectar JSON útil
probe_headers = {
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, text/plain, */*',
    'Referer': URL,
}

probe_results = []
seen = set()
for p in candidates:
    if p in seen:
        continue
    seen.add(p)
    full = urllib.parse.urljoin(BASE, p)
    try:
        req = urllib.request.Request(full, headers=probe_headers)
        with urllib.request.urlopen(req, context=ctx, timeout=20) as r:
            body = r.read()
            ctype = (r.headers.get('Content-Type') or '').lower()
            sample = body[:300].decode('utf-8', errors='replace')
            is_json = 'application/json' in ctype or sample.strip().startswith('{') or sample.strip().startswith('[')
            probe_results.append({
                'url': full,
                'status': r.status,
                'ctype': ctype,
                'is_json': is_json,
                'sample': sample,
                'size': len(body),
            })
    except Exception as e:
        probe_results.append({'url': full, 'error': str(e)})

ok = [x for x in probe_results if x.get('status') == 200]
ok_json = [x for x in ok if x.get('is_json')]
print('total_probes:', len(probe_results))
print('status_200:', len(ok))
print('json_like_200:', len(ok_json))
for x in ok_json[:30]:
    print('\nURL:', x['url'])
    print('ctype:', x['ctype'])
    print('size:', x['size'])
    print('sample:', x['sample'][:200].replace('\n',' '))

In [ ]:
# Buscar estructura de campos de matchup en respuestas JSON
json_hits = []
for x in ok_json:
    u = x['url']
    try:
        req = urllib.request.Request(u, headers=probe_headers)
        with urllib.request.urlopen(req, context=ctx, timeout=20) as r:
            text = r.read().decode('utf-8', errors='replace')
        data = json.loads(text)
        s = text.lower()
        keys_of_interest = ['player', 'entrant', 'participant', 'round', 'match', 'score', 'winner', 'bracket']
        if any(k in s for k in keys_of_interest):
            json_hits.append((u, data))
    except Exception:
        pass

print('json_hits_with_keywords:', len(json_hits))
for u,data in json_hits[:10]:
    print('\nURL:', u)
    if isinstance(data, dict):
        print('top_keys:', list(data.keys())[:30])
    elif isinstance(data, list):
        print('list_len:', len(data), 'item_type:', type(data[0]).__name__ if data else None)
        if data and isinstance(data[0], dict):
            print('item_keys:', list(data[0].keys())[:30])

In [5]:
# Descargar y analizar el JS de pairings-section para ver endpoints exactos
pairings_js_url = 'https://melee.gg/Scripts/Views/Tournament/pairings-section.min.js?v=0.3.63.85&lg=en'
standings_js_url = 'https://melee.gg/Scripts/Views/Tournament/standings-section.min.js?v=0.3.63.85&lg=en'

for label, js_url in [('pairings', pairings_js_url), ('standings', standings_js_url)]:
    try:
        req = urllib.request.Request(js_url, headers={'User-Agent': headers['User-Agent'], 'Referer': URL})
        with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
            js_code = r.read().decode('utf-8', errors='replace')
        print(f'\n=== {label} JS (len={len(js_code)}) ===')
        # Find all URL paths used in ajax/fetch calls
        url_patterns = re.findall(r'["\'](/[A-Za-z0-9_/]+(?:Get|Post|Load|Fetch|Search|List|Round|Pairing|Standing|Match)[A-Za-z0-9_/]*)["\']', js_code, re.I)
        print(f'URL patterns: {url_patterns}')
        # Also look for $.get, $.post, $.ajax, fetch patterns
        ajax_patterns = re.findall(r'(?:url|fetch|get|post|ajax)\s*[\(:]?\s*["\']([^"\']+)["\']', js_code, re.I)
        print(f'Ajax patterns: {set(ajax_patterns)}')
        # Print the full JS for inspection
        print(f'\n--- Full JS ---')
        print(js_code[:5000])
    except Exception as e:
        print(f'Error {label}: {e}')


=== pairings JS (len=1186) ===
URL patterns: []
Ajax patterns: set()

--- Full JS ---
$(()=>{function f(){return $(".round-selector.active",n).data("id")}const r=$("#tournament-pairings-table"),n=$("#pairings-round-selector-container"),i=$(`input[name="DecklistEnabled"]`).val().toLowerCase()==="true",u=[Flattsware.dataTables.column(`TableNumber`,`Table`,!0,!0,!0,(n,t,i)=>i.TableNumberDescription),Flattsware.dataTables.column("PodNumber","Pod Number",!0,!0,!1),Flattsware.dataTables.publicTeamsColumn("Teams","Players/Teams"),Flattsware.dataTables.columns.matchesDecklistColumn("Decklists","Decklists",!1,!1,i),Flattsware.dataTables.column("ResultString","Result",!1,!1,!0),];n.children('[data-is-started="True"]:last').length>0?n.children('[data-is-started="True"]:last').addClass("active"):n.children(":first").addClass("active");const t=Flattsware.dataTables.initTable(r,"b98c0b6d-6d4a-4380-b322-0ae8fd142380",`/Match/GetRoundMatches/${f()}`,"matches",u,{scrollX:!0,order:[0,"asc"]});$(".round

In [6]:
# Extraer IDs de ronda del HTML (están en los round-selector buttons del pairings)
round_selectors = re.findall(r'data-id="([^"]+)"[^>]*class="[^"]*round-selector', html, re.I)
if not round_selectors:
    round_selectors = re.findall(r'class="[^"]*round-selector[^"]*"[^>]*data-id="([^"]+)"', html, re.I)
if not round_selectors:
    # Try broader pattern
    round_selectors = re.findall(r'round-selector[^>]*data-id="([^"]+)"', html, re.I)
if not round_selectors:
    # Extract any data-id near round-related context
    round_selectors = re.findall(r'data-id="([0-9a-f\-]{30,})"', html, re.I)

print(f'round IDs found: {len(round_selectors)}')
for rid in round_selectors[:30]:
    print(rid)

# Also look for the pairings-round-selector-container content
idx = html.find('pairings-round-selector-container')
if idx >= 0:
    snippet = html[idx:idx+3000]
    print('\n--- pairings container snippet ---')
    print(snippet[:2000])

round IDs found: 36
1186217
1186218
1186219
1186220
1186221
1186222
1186223
1186224
1186225
1186226
1186227
1186228
1186229
1186230
1186231
1186232
1186233
1186234
1186217
1186218
1186219
1186220
1186221
1186222
1186223
1186224
1186225
1186226
1186227
1186228

--- pairings container snippet ---
pairings-round-selector-container" class="round-selector-container">
            <button class="btn btn-gray round-selector" data-id="1186217" data-name="Round 1" data-is-started="True">Round 1</button>
            <button class="btn btn-gray round-selector" data-id="1186218" data-name="Round 2" data-is-started="True">Round 2</button>
            <button class="btn btn-gray round-selector" data-id="1186219" data-name="Round 3" data-is-started="True">Round 3</button>
            <button class="btn btn-gray round-selector" data-id="1186220" data-name="Round 4" data-is-started="True">Round 4</button>
            <button class="btn btn-gray round-selector" data-id="1186221" data-name="Round 5" data-

In [7]:
# Probar endpoint /Match/GetRoundMatches con round 1
test_round_id = '1186217'
test_url = f'{BASE}/Match/GetRoundMatches/{test_round_id}'
probe_headers = {
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, text/plain, */*',
    'Referer': URL,
    'X-Requested-With': 'XMLHttpRequest',
}
req = urllib.request.Request(test_url, headers=probe_headers)
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    match_text = r.read().decode('utf-8', errors='replace')
    ctype = r.headers.get('Content-Type')

print('Content-Type:', ctype)
print('Response length:', len(match_text))

match_data = json.loads(match_text)
print('Type:', type(match_data).__name__)
if isinstance(match_data, dict):
    print('Keys:', list(match_data.keys()))
    # Check if there's a 'matches' or 'data' key
    for k, v in match_data.items():
        if isinstance(v, list):
            print(f'  {k}: list of {len(v)} items')
            if v and isinstance(v[0], dict):
                print(f'    First item keys: {list(v[0].keys())}')
                print(f'    First item sample: {json.dumps(v[0], indent=2)[:1000]}')
elif isinstance(match_data, list):
    print(f'List of {len(match_data)} items')
    if match_data and isinstance(match_data[0], dict):
        print(f'First item keys: {list(match_data[0].keys())}')
        print(f'First item: {json.dumps(match_data[0], indent=2)[:1500]}')


Content-Type: text/html; charset=utf-8
Response length: 2498


JSONDecodeError: Expecting value: line 4 column 1 (char 6)

In [8]:
# Ver qué devolvió match_text y probar con DataTables POST format
print('match_text preview:')
print(match_text[:500])
print('\n---\n')

# Looking at the pairings JS, it uses Flattsware.dataTables.initTable which is a DataTables 
# server-side processing setup. The ajax URL is /Match/GetRoundMatches/{roundId}
# DataTables sends POST with draw, start, length, columns[], order[], search params
# Let's try a POST request with DataTables params

import urllib.parse as up

dt_params = {
    'draw': '1',
    'start': '0',
    'length': '-1',  # -1 = all records
    'search[value]': '',
    'search[regex]': 'false',
    'order[0][column]': '0',
    'order[0][dir]': 'asc',
}

post_data = up.urlencode(dt_params).encode('utf-8')
dt_url = f'{BASE}/Match/GetRoundMatches/{test_round_id}'
dt_headers = {
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, text/plain, */*',
    'Referer': URL,
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
}

req2 = urllib.request.Request(dt_url, data=post_data, headers=dt_headers)
with urllib.request.urlopen(req2, context=ctx, timeout=30) as r2:
    dt_text = r2.read().decode('utf-8', errors='replace')
    dt_ctype = r2.headers.get('Content-Type')

print('POST Content-Type:', dt_ctype)
print('POST Response length:', len(dt_text))
print('POST Response preview:')
print(dt_text[:2000])

match_text preview:



<div class="content page-with-container">
    <div class="container text-center pt-5 pb-5">
        <div class="d-flex flex-row flex-gap-1 mx-auto" style="max-width: 600px;">
            <div class="d-flex flex-column text-left flex-grow-1" style="gap: 1rem;">
                <h1>Something's wrong here...</h1>
                <div>It looks like something went wrong with your last request. Check out our help center or go back to where you were.</div>
                <div class="d-flex

---



HTTPError: HTTP Error 500: Internal Server Error

In [9]:
# Analizar flattsware.datatables.min.js para entender cómo hace los requests
dt_js_url = 'https://melee.gg/Scripts/flattsware.datatables.min.js?v=0.3.63.85&lg=en'
req = urllib.request.Request(dt_js_url, headers={'User-Agent': headers['User-Agent'], 'Referer': URL})
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    dt_js = r.read().decode('utf-8', errors='replace')

print(f'datatables JS length: {len(dt_js)}')

# Search for initTable function and ajax setup
# Look for key patterns: initTable, ajax, url, request verification
for keyword in ['initTable', 'RequestVerification', 'anti-forgery', 'token', '__RequestVerification', 'ajax', 'dataSrc', 'serverSide']:
    positions = [m.start() for m in re.finditer(keyword, dt_js, re.I)]
    if positions:
        print(f'\n=== "{keyword}" found at {len(positions)} positions ===')
        for pos in positions[:3]:
            start = max(0, pos - 50)
            end = min(len(dt_js), pos + 200)
            print(f'  ...{dt_js[start:end]}...')


datatables JS length: 29901

=== "initTable" found at 7 positions ===
  ...e`,`100 per page`,`250 per page`,`500 per page`]],initTable:function(n,t,i,r,u,f){const e=Flattsware.dataTables.defaultOptions(t,i,r,u,f);e.disableLoadingRows===!0||(e.preDrawCallback=e.eightLoadingRows===!0?function(){$("tbody",n).each(function(){$(...
  ...);return n.defaultOptions=e,Flattsware.dataTables.initTableElements(n),o},dom:function(){return`<"datatables-controls-row"><"d-flex flex-wrap datatables-filters-row">t<"row"<"col-12 col-md-6"i><"col-12 col-md-6"p>>r`},language:function(n){let t;retur...
  ...se(localStorage.getItem(`DataTables_${n}`))}},u)},initTableElements:function(n){Flattsware.dataTables.addTableElements(n);Flattsware.dataTables.initRefreshButton(n);Flattsware.dataTables.initTableEditors(n);Flattsware.dataTables.initDebounce(n);Flatt...

=== "ajax" found at 11 positions ===
  ...nsive:function(){return{details:{type:"column"}}},ajax:function(n,t,i,r){return{cache:!0,url:n,type:"POST",dat

In [10]:
# Check for anti-forgery tokens in HTML
token_patterns = [
    r'__RequestVerificationToken[^"]*"[^"]*value="([^"]+)"',
    r'name="__RequestVerificationToken"\s+value="([^"]+)"',
    r'value="([^"]+)"\s+name="__RequestVerificationToken"',
    r'RequestVerificationToken["\s:]+([^"]+)',
    r'antiforgery["\s:]+([^"]+)',
]
for pat in token_patterns:
    found = re.findall(pat, html, re.I)
    if found:
        print(f'FOUND token with pattern: {pat[:50]}...')
        print(f'  Value: {found[0][:80]}...')

# Also check for any hidden inputs
hidden_inputs = re.findall(r'<input[^>]+type=["\']hidden["\'][^>]+>', html, re.I)
print(f'\nHidden inputs: {len(hidden_inputs)}')
for h in hidden_inputs[:20]:
    print(f'  {h[:200]}')

# Check cookies / set-cookie in response  
print('\nResponse headers:')
for k, v in response_headers.items():
    print(f'  {k}: {v[:100]}')

# Now try proper DataTables POST with full column definitions
import urllib.request, urllib.parse

# Use http.cookiejar to handle cookies properly
import http.cookiejar
cj = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(
    urllib.request.HTTPCookieProcessor(cj),
    urllib.request.HTTPSHandler(context=ctx)
)

# First GET the page to collect cookies
page_req = urllib.request.Request(URL, headers={
    'User-Agent': headers['User-Agent'],
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
})
with opener.open(page_req, timeout=30) as resp:
    page_html = resp.read().decode('utf-8', errors='replace')

print(f'\nCookies after GET: {len(list(cj))}')
for c in cj:
    print(f'  {c.name}={c.value[:50]}...')

# Build DataTables POST params for Round 1
dt_post = {
    'draw': '1',
    'start': '0',
    'length': '-1',
    'search[value]': '',
    'search[regex]': 'false',
    'order[0][column]': '0',
    'order[0][dir]': 'asc',
    'columns[0][data]': 'TableNumber',
    'columns[0][name]': 'TableNumber',
    'columns[0][searchable]': 'true',
    'columns[0][orderable]': 'true',
    'columns[1][data]': 'PodNumber',
    'columns[1][name]': 'PodNumber',
    'columns[1][searchable]': 'true',
    'columns[1][orderable]': 'true',
    'columns[2][data]': 'Teams',
    'columns[2][name]': 'Teams',
    'columns[2][searchable]': 'false',
    'columns[2][orderable]': 'false',
    'columns[3][data]': 'Decklists',
    'columns[3][name]': 'Decklists',
    'columns[3][searchable]': 'false',
    'columns[3][orderable]': 'false',
    'columns[4][data]': 'ResultString',
    'columns[4][name]': 'ResultString',
    'columns[4][searchable]': 'false',
    'columns[4][orderable]': 'false',
}

post_data = urllib.parse.urlencode(dt_post).encode('utf-8')
match_url = f'{BASE}/Match/GetRoundMatches/{test_round_id}'
match_req = urllib.request.Request(match_url, data=post_data, headers={
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, text/plain, */*',
    'Referer': URL,
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
})

try:
    with opener.open(match_req, timeout=30) as r:
        match_resp = r.read().decode('utf-8', errors='replace')
        print(f'\nMatch POST status: {r.status}')
        print(f'Content-Type: {r.headers.get("Content-Type")}')
        print(f'Length: {len(match_resp)}')
        print(f'Preview: {match_resp[:2000]}')
except Exception as e:
    print(f'Error: {e}')
    if hasattr(e, 'read'):
        err_body = e.read().decode('utf-8', errors='replace')
        print(f'Error body: {err_body[:500]}')


Hidden inputs: 7
  <input id="current-language" type="hidden" value="English"/>
  <input id="current-language-tag" type="hidden" value="en"/>
  <input id="tournament-id" name="tournament-id" type="hidden" value="339227" />
  <input id="TournamentGuid" name="TournamentGuid" type="hidden" value="8de846e6-8a92-404c-bc8e-ee04c4173169" />
  <input id="IsCheckInEnabled" name="IsCheckInEnabled" type="hidden" value="True" />
  <input data-val="true" data-val-required="The DecklistEnabled field is required." id="DecklistEnabled" name="DecklistEnabled" type="hidden" value="true" />
  <input id="Via" name="Via" type="hidden" value="TournamentView" />

Response headers:
  Date: Mon, 23 Feb 2026 16:08:03 GMT
  Content-Type: text/html; charset=utf-8
  Content-Length: 124630
  Connection: close
  Cache-Control: private
  Server: Microsoft-IIS/10.0
  Strict-Transport-Security: max-age=63072000; includeSubdomains; preload
  Referrer-Policy: no-referrer
  X-Frame-Options: DENY
  X-Content-Type-Options:

In [11]:
# Check flattsware.core.min.js for global AJAX setup (ajaxSetup, beforeSend, headers)
core_js_url = 'https://melee.gg/Scripts/flattsware.core.min.js?v=0.3.63.85&lg=en'
req = urllib.request.Request(core_js_url, headers={'User-Agent': headers['User-Agent'], 'Referer': URL})
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    core_js = r.read().decode('utf-8', errors='replace')

print(f'core JS length: {len(core_js)}')

# Search for global AJAX configuration
for kw in ['ajaxSetup', 'beforeSend', 'ajaxPrefilter', 'X-Requested', 'headers', 'csrf', 'token', 'Authorization']:
    positions = [m.start() for m in re.finditer(kw, core_js, re.I)]
    if positions:
        print(f'\n=== "{kw}" at {len(positions)} positions ===')
        for pos in positions[:5]:
            s = max(0, pos - 80)
            e = min(len(core_js), pos + 300)
            print(f'  ...{core_js[s:e]}...\n')


core JS length: 19379

=== "token" at 12 positions ===
  ...("="),n[0]&&(r[n[0]]=n[1]===undefined?!0:decodeURIComponent(n[1]));return r},getTokenPromise:()=>new Promise((n,t)=>{if(Flattsware.inMemoryToken.Expires&&moment(Flattsware.inMemoryToken.Expires)>moment.utc()){n(Flattsware.inMemoryToken);return}$.get("/Home/RefreshToken",t=>{Flattsware.inMemoryToken=t,n(Flattsware.inMemoryToken)}).fail(()=>{Flattsware.showDefaultFailMessage(),co...

  ...(n[1]));return r},getTokenPromise:()=>new Promise((n,t)=>{if(Flattsware.inMemoryToken.Expires&&moment(Flattsware.inMemoryToken.Expires)>moment.utc()){n(Flattsware.inMemoryToken);return}$.get("/Home/RefreshToken",t=>{Flattsware.inMemoryToken=t,n(Flattsware.inMemoryToken)}).fail(()=>{Flattsware.showDefaultFailMessage(),console.error(`There was an error fetching an in-memory token...

  ... Promise((n,t)=>{if(Flattsware.inMemoryToken.Expires&&moment(Flattsware.inMemoryToken.Expires)>moment.utc()){n(Flattsware.inMemoryToken);return}$.get("/Home/Ref

In [12]:
# Check how inMemoryToken is used in DataTables AJAX calls
for kw in ['inMemoryToken', 'getToken', 'Token', 'beforeSend', 'Authorization', 'Bearer']:
    positions = [m.start() for m in re.finditer(kw, dt_js, re.I)]
    if positions:
        print(f'\n=== DataTables JS: "{kw}" at {len(positions)} positions ===')
        for pos in positions[:5]:
            s = max(0, pos - 100)
            e = min(len(dt_js), pos + 300)
            print(f'  ...{dt_js[s:e]}...\n')

# Now try /Home/RefreshToken
print('\n=== Trying /Home/RefreshToken ===')
token_url = f'{BASE}/Home/RefreshToken'
req = urllib.request.Request(token_url, headers={
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, text/plain, */*',
    'Referer': URL,
})
try:
    with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
        token_text = r.read().decode('utf-8', errors='replace')
        print(f'Status: {r.status}')
        print(f'Content-Type: {r.headers.get("Content-Type")}')
        print(f'Response: {token_text[:500]}')
except Exception as e:
    print(f'Error: {e}')
    if hasattr(e, 'read'):
        print(f'Body: {e.read().decode("utf-8", errors="replace")[:500]}')



=== Trying /Home/RefreshToken ===
Status: 200
Content-Type: text/html; charset=utf-8
Response: <!DOCTYPE html>
<html>
<head>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"/>

<title>Sign In to Your Account | Melee</title>
<impact-site-verification value="4c243f18-8267-493f-a951-c8659ec151bc"></impact-site-verification>
<meta content="@MeleePlatform" property="twitter:site"></meta>
<meta content="0.3.63.85" name="version"></meta>
<meta content="https://cdn.melee.gg/im


In [13]:
# Full DataTables POST with all required sub-params
import urllib.parse as up

dt_full = {}
dt_full['draw'] = '1'
dt_full['start'] = '0'
dt_full['length'] = '25'
dt_full['search[value]'] = ''
dt_full['search[regex]'] = 'false'
dt_full['order[0][column]'] = '0'
dt_full['order[0][dir]'] = 'asc'

# 5 columns matching the pairings JS
cols = [
    ('TableNumber', 'true', 'true'),
    ('PodNumber', 'true', 'true'),
    ('Teams', 'false', 'false'),
    ('Decklists', 'false', 'false'),
    ('ResultString', 'false', 'false'),
]
for i, (data, searchable, orderable) in enumerate(cols):
    dt_full[f'columns[{i}][data]'] = data
    dt_full[f'columns[{i}][name]'] = ''
    dt_full[f'columns[{i}][searchable]'] = searchable
    dt_full[f'columns[{i}][orderable]'] = orderable
    dt_full[f'columns[{i}][search][value]'] = ''
    dt_full[f'columns[{i}][search][regex]'] = 'false'

post_bytes = up.urlencode(dt_full).encode('utf-8')
match_url = f'{BASE}/Match/GetRoundMatches/{test_round_id}'

print(f'POST URL: {match_url}')
print(f'POST body: {up.urlencode(dt_full)[:500]}')

req = urllib.request.Request(match_url, data=post_bytes, headers={
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Referer': URL,
    'Origin': 'https://melee.gg',
})
try:
    with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
        resp_text = r.read().decode('utf-8', errors='replace')
        print(f'Status: {r.status}')
        print(f'Content-Type: {r.headers.get("Content-Type")}')
        print(f'Length: {len(resp_text)}')
        print(f'Preview: {resp_text[:3000]}')
except Exception as e:
    print(f'Error: {e}')
    if hasattr(e, 'read'):
        err_body = e.read().decode('utf-8', errors='replace')
        print(f'Error body: {err_body[:500]}')
    
    # Try without X-Requested-With and with different approach
    print('\n--- Retry without X-Requested-With ---')
    req2 = urllib.request.Request(match_url, data=post_bytes, headers={
        'User-Agent': headers['User-Agent'],
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Referer': URL,
        'Origin': 'https://melee.gg',
    })
    try:
        with urllib.request.urlopen(req2, context=ctx, timeout=30) as r:
            resp_text = r.read().decode('utf-8', errors='replace')
            print(f'Status: {r.status}')
            print(f'Preview: {resp_text[:2000]}')
    except Exception as e2:
        print(f'Error2: {e2}')
        if hasattr(e2, 'read'):
            print(f'Body2: {e2.read().decode("utf-8", errors="replace")[:500]}')

POST URL: https://melee.gg/Match/GetRoundMatches/1186217
POST body: draw=1&start=0&length=25&search%5Bvalue%5D=&search%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&columns%5B0%5D%5Bdata%5D=TableNumber&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=PodNumber&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bv
Status: 200
Content-Type: application/json; charset=utf-8
Length: 53322
Preview: {"draw":1,"recordsTotal":597,"recordsFiltered":597,"data":[{"Competitors":[{"Team":{"Players":[{"TeamId":3819821,"ID":3805157,"ScreenName":"N/A","ProfileImageVersion":0,"DisplayName":"Avi Grinberg","DisplayNameLastFirst":"Grinberg, Avi","Username":"popohad","LanguageDescription":"Auto (English)","PronounsDescription":null}],"ID":3819821,"Name":null

In [14]:
# Ejecutar el scraper completo
import importlib, sys
sys.path.insert(0, '/workspaces/scrapermeleegg')

# Importar y ejecutar
import melee_scraper as ms
importlib.reload(ms)

tournament = ms.scrape_tournament('339227')


[*] Descargando página del torneo 339227...
[*] Torneo: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
[*] Rondas encontradas: 18
  [>] Round 1 (id=1186217)... 597 matches
  [>] Round 2 (id=1186218)... 586 matches
  [>] Round 3 (id=1186219)... 582 matches
  [>] Round 4 (id=1186220)... 565 matches
  [>] Round 5 (id=1186221)... 531 matches
  [>] Round 6 (id=1186222)... 468 matches
  [>] Round 7 (id=1186223)... 397 matches
  [>] Round 8 (id=1186224)... 318 matches
  [>] Round 9 (id=1186225)... 249 matches
  [>] Round 10 (id=1186226)... 140 matches
  [>] Round 11 (id=1186227)... 136 matches
  [>] Round 12 (id=1186228)... 132 matches
  [>] Round 13 (id=1186229)... 122 matches
  [>] Round 14 (id=1186230)... 112 matches
  [>] Round 15 (id=1186231)... 104 matches
  [>] Quarterfinals (id=1186232)... 4 matches
  [>] Semifinals (id=1186233)... 2 matches
  [>] Finals (id=1186234)... 1 matches

[*] Total: 5046 matches, 1193 jugadores


In [15]:
# Construir y mostrar la matriz de emparejamientos
matrix_data = ms.build_matchup_matrix(tournament)
ms.print_matchup_matrix(matrix_data)



MATRIZ DE EMPAREJAMIENTOS (Win-Loss-Draw)

Deck                                   W    L    D  Total    Win%
-----------------------------------------------------------------
Dimir Combo                            1    0    0      1  100.0%
Gruul Harmonizer                      10    5    0     15   66.7%
Four-Color Rhythm                     31   20    0     51   60.8%
Izzet Prowess                        253  163    5    421   60.1%
Jeskai Dragons                         9    6    0     15   60.0%
Sultai Control                         9    4    2     15   60.0%
Azorius Otters                        29   21    0     50   58.0%
Grixis Monument                       12    9    0     21   57.1%
Temur Rhythm                          39   30    1     70   55.7%
Mono-Green Landfalls                   5    4    0      9   55.6%
Orzhov Tokens                          5    3    1      9   55.6%
Selesnya Rhythm                        5    4    0      9   55.6%
Temur Elementals                

In [16]:
# Exportar datos
import os
os.chdir('/workspaces/scrapermeleegg')

ms.export_matches_csv(tournament, 'matches.csv')
ms.export_matrix_csv(matrix_data, 'matchup_matrix.csv')
ms.export_json(tournament, matrix_data, 'tournament_data.json')

# Resumen rápido
print(f'\nDecks únicos: {len(matrix_data["decks"])}')
print(f'Matches con decklist: {len(matrix_data["player_matches"])}')

# Top 15 decks por cantidad de partidas
from collections import Counter
deck_counts = Counter()
for m in matrix_data['player_matches']:
    deck_counts[m['player1_deck']] += 1
    deck_counts[m['player2_deck']] += 1
    
print(f'\nTop 15 decks (por apariciones en matches):')
for deck, count in deck_counts.most_common(15):
    print(f'  {deck}: {count} partidas')

[*] Matches exportados a: matches.csv
[*] Matriz exportada a: matchup_matrix.csv
[*] Datos completos exportados a: tournament_data.json

Decks únicos: 107
Matches con decklist: 5036

Top 15 decks (por apariciones en matches):
  Izzet Lessons: 1159 partidas
  Mono-Green Landfall: 1026 partidas
  Dimir Excruciator: 824 partidas
  Dimir Midrange: 684 partidas
  Simic Rhythm: 644 partidas
  Azorius Tempo: 639 partidas
  Izzet Elementals: 479 partidas
  Izzet Spellementals: 459 partidas
  Izzet Prowess: 421 partidas
  Boros Dragons: 364 partidas
  Jeskai Control: 295 partidas
  Temur Harmonizer: 279 partidas
  Bant Airbending: 274 partidas
  Mono-Red Aggro: 255 partidas
  Rakdos Monument: 138 partidas


In [17]:
# Investigar endpoint de decklists - descargar flattsware.decklist.min.js
decklist_js_url = 'https://melee.gg/Scripts/flattsware.decklist.min.js?v=0.3.63.85&lg=en'
req = urllib.request.Request(decklist_js_url, headers={'User-Agent': headers['User-Agent'], 'Referer': URL})
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    decklist_js = r.read().decode('utf-8', errors='replace')

print(f'decklist JS length: {len(decklist_js)}')
print(f'\n--- Full JS ---')
print(decklist_js[:5000])

# Search for URL patterns
import re
url_pats = re.findall(r'["\'](/[A-Za-z0-9_/]+(?:Deck|decklist|List)[A-Za-z0-9_/]*)["\']', decklist_js, re.I)
print(f'\nURL patterns with Deck/decklist/List: {url_pats}')

# Search for any endpoint-like strings
all_paths = re.findall(r'["\'](/[A-Za-z]+/[A-Za-z]+)["\']', decklist_js)
print(f'\nAll path patterns: {set(all_paths)}')

decklist JS length: 2538

--- Full JS ---
var Flattsware=Flattsware||{};Flattsware.decklist={initFormatExplanationButtons:function(){const n=$("#decklist-format-explanation-modal");$(document).on("click",".decklist-format-explanation-button",function(t){t.preventDefault();n.data("format-id",$(this).data("format-id"));n.modal("show")});n.on("shown.bs.modal",function(){const t=n.find(".modal-body"),i=n.data("format-id"),r=n.data("last-format-id");i!==r&&(n.data("last-format-id",i),Flattsware.spinner.initContainer(t),$.get(`/Format/GetFormatConfig/${i}`,function(n){Flattsware.notify.handleResponse(n,function(n){const i=JSON.parse(n.Json);t.html(Flattsware.formats.configExplanationHtml(i))})}).fail(function(n){Flattsware.showFail(n);t.html('<div class="alert alert-danger">There was an error loading the format explanation.<\/div>')}))})},splitInputName:function(n){const i=typeof n=="string"?n:"",t=i.split("|"),r=$.trim(t[0]),u=t.length>1?$.trim(t[1]):null;return[r,u]},sanitizeName:function(

In [18]:
# Buscar endpoint de decklists en el HTML y JS
# Ya sabemos que los matches incluyen DecklistId como UUID
# Buscar en HTML links a decklists
decklist_links = re.findall(r'href="([^"]*[Dd]ecklist[^"]*)"', html, re.I)
print(f'Decklist links in HTML: {decklist_links[:10]}')

# Buscar en todos los JS descargados
for u, js in js_texts.items():
    paths = re.findall(r'["\'](/[A-Za-z]+/[A-Za-z]+(?:Decklist|GetDecklist|ViewDecklist|DecklistDetail|GetCards)[A-Za-z/]*)["\']', js, re.I)
    if paths:
        print(f'\n{u}:')
        for p in paths:
            print(f'  {p}')

# Try common patterns for decklist viewing
# melee.gg typically uses /Decklist/View/{guid} or /Decklist/GetDecklistCards/{guid}
sample_deck_id = 'dfc40304-07a5-450b-8ee6-b3f700eff6ee'  # From Avi Grinberg's "Simic Rhythm"

attempts = [
    f'/Decklist/View/{sample_deck_id}',
    f'/Decklist/GetDecklistCards/{sample_deck_id}',
    f'/Decklist/GetDecklist/{sample_deck_id}',
    f'/Decklist/Get/{sample_deck_id}',
    f'/Decklist/{sample_deck_id}',
    f'/Decklist/GetDecklistDetail/{sample_deck_id}',
]

for path in attempts:
    full_url = f'{BASE}{path}'
    try:
        req = urllib.request.Request(full_url, headers={
            'User-Agent': headers['User-Agent'],
            'Accept': 'text/html,application/json,*/*',
            'Referer': URL,
        })
        with urllib.request.urlopen(req, context=ctx, timeout=15) as r:
            body = r.read().decode('utf-8', errors='replace')
            print(f'\n[OK] {path} -> status={r.status}, len={len(body)}, ctype={r.headers.get("Content-Type")}')
            print(f'  Preview: {body[:500]}')
    except Exception as e:
        code = getattr(e, 'code', '?')
        print(f'[{code}] {path}')

Decklist links in HTML: ['/Decklist/View/{{DecklistId}}', '/Decklist/View/{{DecklistId}}', '/Decklist/View/{{DecklistId}}', '/Decklist/View/{{Guid}}', '/Decklist/View/{{Decklist.ID}}', '/Decklist/View/{{id}}', '/Decklist/View/{{opponentDecklistId}}', '{{Decklist.ScreenshotUrl}}', '/Decklists', '/Decklists']

https://melee.gg/Scripts/flattsware.hovercard.min.js?v=0.3.63.85&lg=en:
  /Decklist/GetDecklistDetails
  /Decklist/GetDecklistComparison

[OK] /Decklist/View/dfc40304-07a5-450b-8ee6-b3f700eff6ee -> status=200, len=106914, ctype=text/html; charset=utf-8
  Preview: <!DOCTYPE html>
<html>
<head>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"/>

<title>Simic Rhythm | Melee</title>
<impact-site-verification value="4c243f18-8267-493f-a951-c8659ec151bc"></impact-site-verification>
<meta content="@MeleePlatform" property="twitter:site"></meta>
<meta content="0.3.63.85" name="ve

In [19]:
# Analizar /Decklist/View/{id} para extraer cartas del HTML
sample_deck_id = 'dfc40304-07a5-450b-8ee6-b3f700eff6ee'
deck_url = f'{BASE}/Decklist/View/{sample_deck_id}'
req = urllib.request.Request(deck_url, headers={'User-Agent': headers['User-Agent'], 'Accept': 'text/html'})
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    deck_html = r.read().decode('utf-8', errors='replace')

print(f'Deck HTML length: {len(deck_html)}')

# Look for card data patterns in the HTML
# Check for inline JSON with card data
inline_scripts = re.findall(r'<script[^>]*>(.*?)</script>', deck_html, re.DOTALL | re.I)
for i, sc in enumerate(inline_scripts):
    if len(sc.strip()) > 50 and any(kw in sc.lower() for kw in ['card', 'deck', 'main', 'side', 'companion', 'quantity']):
        print(f'\n--- Script {i} (len={len(sc)}) ---')
        print(sc[:1000])

# Look for decklist sections in HTML
for pattern in [r'class="[^"]*decklist[^"]*"', r'class="[^"]*card-name[^"]*"', r'class="[^"]*deck-card[^"]*"',
                r'class="[^"]*mainboard[^"]*"', r'class="[^"]*sideboard[^"]*"']:
    matches = re.findall(pattern, deck_html, re.I)
    if matches:
        print(f'\nPattern "{pattern}": {len(matches)} matches')
        print(f'  First: {matches[0]}')

Deck HTML length: 106914

--- Script 1 (len=7371) ---
    
    {{#quickLookup}}
    <div class="data-tables-quick-filter-input-wrapper">
        <input id="{{tableId}}-quick-filter-input" class="form-control data-tables-quick-filter-input" placeholder="Quick search..." aria-quicksearch="{{tableId}}"/>
    </div>
    {{/quickLookup}}
    
    <div class="data-tables-filter-input-wrapper">
        <input id="{{tableId}}-filter-input" class="form-control data-tables-filter-input" placeholder="Search results..." aria-controls="{{tableId}}"/>
    </div>
    {{#useFilters}}
    <div class="dropdown data-tables-filter-dropdown data-tables-filter-dropdown-wrapper" id="accordion-{{tableId}}">
        <button class="btn btn-secondary dropdown-toggle data-tables-filter-dropdown-button" type="button" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">Filter</button>
        <div class="dropdown-menu data-tables-filter-dropdown m-0 p-0">
            {{#filters}}
            <div clas

In [20]:
# Try /Decklist/GetDecklistDetails as JSON POST (found in hovercard JS)
deck_details_url = f'{BASE}/Decklist/GetDecklistDetails'
# Try POST with decklistId
import urllib.parse as up
post_data = up.urlencode({'decklistId': sample_deck_id}).encode('utf-8')
req = urllib.request.Request(deck_details_url, data=post_data, headers={
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, */*',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Referer': deck_url,
    'Origin': BASE,
})
try:
    with urllib.request.urlopen(req, context=ctx, timeout=15) as r:
        details_text = r.read().decode('utf-8', errors='replace')
        print(f'Status: {r.status}, ctype: {r.headers.get("Content-Type")}, len: {len(details_text)}')
        print(f'Preview: {details_text[:2000]}')
except Exception as e:
    print(f'POST Error: {e}')
    # Try GET
    get_url = f'{deck_details_url}/{sample_deck_id}'
    req2 = urllib.request.Request(get_url, headers={
        'User-Agent': headers['User-Agent'],
        'Accept': 'application/json, */*',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': deck_url,
    })
    try:
        with urllib.request.urlopen(req2, context=ctx, timeout=15) as r:
            details_text = r.read().decode('utf-8', errors='replace')
            print(f'GET Status: {r.status}, len: {len(details_text)}')
            print(f'Preview: {details_text[:2000]}')
    except Exception as e2:
        print(f'GET Error: {e2}')

Status: 200, ctype: text/html; charset=utf-8, len: 2498
Preview: 


<div class="content page-with-container">
    <div class="container text-center pt-5 pb-5">
        <div class="d-flex flex-row flex-gap-1 mx-auto" style="max-width: 600px;">
            <div class="d-flex flex-column text-left flex-grow-1" style="gap: 1rem;">
                <h1>Something's wrong here...</h1>
                <div>It looks like something went wrong with your last request. Check out our help center or go back to where you were.</div>
                <div class="d-flex flex-row flex-gap-1 mt-3">
                    <a href="https://help.melee.gg/docs/contact-melee/" class="btn btn-warning px-3 py-2">Help Center</a>
                    <button role="button" id="error-page-close-button" class="btn btn-outline-warning px-3 py-2" aria-label="Go Back">Go Back</button>
                </div>
            </div>
            <i class="fa-duotone fa-hexagon-exclamation text-danger my-auto" style="font-size: 10rem;

In [21]:
# Analizar flattsware.hovercard.min.js para ver cómo llama a GetDecklistDetails
hovercard_js_url = 'https://melee.gg/Scripts/flattsware.hovercard.min.js?v=0.3.63.85&lg=en'
req = urllib.request.Request(hovercard_js_url, headers={'User-Agent': headers['User-Agent']})
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    hc_js = r.read().decode('utf-8', errors='replace')

print(f'hovercard JS length: {len(hc_js)}')

# Find GetDecklistDetails context
for kw in ['GetDecklistDetails', 'GetDecklistComparison', 'decklistId', 'DecklistId']:
    positions = [m.start() for m in re.finditer(kw, hc_js)]
    if positions:
        print(f'\n=== "{kw}" ===')
        for pos in positions[:3]:
            s = max(0, pos - 150)
            e = min(len(hc_js), pos + 300)
            print(f'  ...{hc_js[s:e]}...\n')

# Also check the decklist view page HTML for JS that loads card data
deck_scripts = re.findall(r'<script[^>]+src=["\']([^"\']+)["\']', deck_html, re.I)
deck_only_scripts = [s for s in deck_scripts if 'decklist' in s.lower() or 'deck' in s.lower()]
print(f'\nDecklist page scripts with "deck": {deck_only_scripts}')

# Search for card data in deck_html more aggressively
# Look for quantity patterns like "4 Card Name" or card sections
card_sections = re.findall(r'decklist-card-(?:name|quantity|section)', deck_html, re.I)
print(f'\nDecklist card class patterns: {len(card_sections)}')

# Try to find the section with card entries
idx1 = deck_html.find('decklist-builder')
if idx1 < 0:
    idx1 = deck_html.find('decklist-section')
if idx1 < 0:
    idx1 = deck_html.find('Mainboard')
if idx1 < 0:
    idx1 = deck_html.find('Main Deck')
if idx1 >= 0:
    print(f'\nFound deck section at {idx1}:')
    print(deck_html[idx1:idx1+2000])

hovercard JS length: 7132

=== "GetDecklistDetails" ===
  ...-type="tournament" data-id="${n}" href="/Tournament/View/${n}">`+`${t}<i class="fas fa-info-circle fa-fw ml-2"></i>`+`</a>`},decklist:{url:"/Decklist/GetDecklistDetails",parameterName:"id"},supportTicket:{url:"/Ticket/GetTicketDetails",parameterName:"id"},compareDecklist:{url:"/Decklist/GetDecklistComparison"},venue:{url:"/Venue/GetDetails",parameterName:"id"}};$(()=>{function i(n,t,i){const r=Flattsware.hovercard[t],f=r.titleTemplate?Mustache.re...


=== "GetDecklistComparison" ===
  ...:"/Decklist/GetDecklistDetails",parameterName:"id"},supportTicket:{url:"/Ticket/GetTicketDetails",parameterName:"id"},compareDecklist:{url:"/Decklist/GetDecklistComparison"},venue:{url:"/Venue/GetDetails",parameterName:"id"}};$(()=>{function i(n,t,i){const r=Flattsware.hovercard[t],f=r.titleTemplate?Mustache.render(r.titleTemplate,i):"",e=Mustache.render(r.bodyTemplate,i),u={container:"body",template:'<div class="popover hovercard" role="tool

In [22]:
# Probar /Decklist/GetDecklistDetails?id={guid} como GET con X-Requested-With
details_url = f'{BASE}/Decklist/GetDecklistDetails?id={sample_deck_id}'
req = urllib.request.Request(details_url, headers={
    'User-Agent': headers['User-Agent'],
    'Accept': 'application/json, */*',
    'X-Requested-With': 'XMLHttpRequest',
    'Referer': f'{BASE}/Decklist/View/{sample_deck_id}',
})
try:
    with urllib.request.urlopen(req, context=ctx, timeout=15) as r:
        details_text = r.read().decode('utf-8', errors='replace')
        print(f'Status: {r.status}, ctype: {r.headers.get("Content-Type")}, len: {len(details_text)}')
        if details_text.strip().startswith('{') or details_text.strip().startswith('['):
            d = json.loads(details_text)
            print(f'JSON parsed! Type: {type(d).__name__}')
            if isinstance(d, dict):
                print(f'Keys: {list(d.keys())}')
                print(json.dumps(d, indent=2)[:3000])
        else:
            print(f'Preview: {details_text[:1000]}')
except Exception as e:
    print(f'Error: {e}')

# Also try decklist.view.min.js to see how it loads card data
dv_js_url = 'https://melee.gg/Scripts/Views/Decklist/decklist.view.min.js?v=0.3.63.85&lg=en'
req = urllib.request.Request(dv_js_url, headers={'User-Agent': headers['User-Agent']})
with urllib.request.urlopen(req, context=ctx, timeout=30) as r:
    dv_js = r.read().decode('utf-8', errors='replace')
print(f'\ndecklist.view JS length: {len(dv_js)}')
print(dv_js[:3000])

Status: 200, ctype: application/json; charset=utf-8, len: 5355
JSON parsed! Type: dict
Keys: ['Guid', 'DecklistName', 'FormatName', 'Game', 'Records', 'LinkToCards', 'Components']
{
  "Guid": "dfc40304-07a5-450b-8ee6-b3f700eff6ee",
  "DecklistName": "Simic Rhythm",
  "FormatName": "Standard",
  "Game": "MagicTheGathering",
  "Records": [
    {
      "l": "archdruidscharm",
      "n": "Archdruid's Charm",
      "s": null,
      "q": 2,
      "c": 0,
      "t": "Instant"
    },
    {
      "l": "badgermolecub",
      "n": "Badgermole Cub",
      "s": null,
      "q": 4,
      "c": 0,
      "t": "Creature"
    },
    {
      "l": "botanicalsanctum",
      "n": "Botanical Sanctum",
      "s": null,
      "q": 4,
      "c": 0,
      "t": "Land"
    },
    {
      "l": "breedingpool",
      "n": "Breeding Pool",
      "s": null,
      "q": 4,
      "c": 0,
      "t": "Land"
    },
    {
      "l": "craterhoofbehemoth",
      "n": "Craterhoof Behemoth",
      "s": null,
      "q": 1,
      "c

In [23]:
import subprocess
result = subprocess.run(['curl', '-sL', 'https://j6e.me/mtg-meta-analyzer/metagame'], capture_output=True, text=True, timeout=30)
html = result.stdout
print(f"Total length: {len(html)} chars, {html.count(chr(10))} lines")
print("=== FIRST 8000 CHARS ===")
print(html[:8000])

Total length: 1559 chars, 41 lines
=== FIRST 8000 CHARS ===
<!doctype html>
<html lang="en">
	<head>
		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width, initial-scale=1" />
		<link href="./_app/immutable/entry/start.DfdyMplu.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/C3NQ9sqC.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/C6U0rBQQ.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/CaosO92D.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/D-luxm3l.js" rel="modulepreload">
		<link href="./_app/immutable/entry/app.BvaApAJx.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/B9QYiEl2.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/Cqu7ILzR.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/D-GAYjSq.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/C6kBW4ne.js" rel="modulepreload">
		<link href="./_app/immutable/chunks/BpeGWlSU.js" rel="modulepreload">

In [24]:
import subprocess, re

# Get the app entry point JS
base = "https://j6e.me/mtg-meta-analyzer"

# Fetch the start and app JS bundles
js_files = [
    f"{base}/_app/immutable/entry/start.DfdyMplu.js",
    f"{base}/_app/immutable/entry/app.BvaApAJx.js",
]

for url in js_files:
    print(f"\n{'='*60}")
    print(f"FILE: {url.split('/')[-1]}")
    print('='*60)
    r = subprocess.run(['curl', '-sL', url], capture_output=True, text=True, timeout=15)
    print(r.stdout[:3000])
    print(f"... (total {len(r.stdout)} chars)")



FILE: start.DfdyMplu.js
import{a as r}from"../chunks/C3NQ9sqC.js";import{w as t}from"../chunks/CaosO92D.js";export{t as load_css,r as start};

... (total 118 chars)

FILE: app.BvaApAJx.js
const __vite__mapDeps=(i,m=__vite__mapDeps,d=(m.f||(m.f=["../nodes/0.CORkh5yo.js","../chunks/Cqu7ILzR.js","../chunks/C6U0rBQQ.js","../chunks/B9QYiEl2.js","../chunks/BcUPtRvI.js","../chunks/C6kBW4ne.js","../chunks/CGzraLuI.js","../chunks/Dm92gg5j.js","../chunks/CicpDTVh.js","../chunks/CaosO92D.js","../chunks/D-luxm3l.js","../chunks/B9yUJoIP.js","../chunks/C3NQ9sqC.js","../assets/0.BV4fSZMG.css","../nodes/1.BcIN0nm1.js","../chunks/pK3uJ45f.js","../chunks/fFS9_pHI.js","../assets/1.DeB8wVPn.css","../nodes/2.C_J5Tjfs.js","../assets/2.DoU2b4SV.css","../nodes/3.vN4Txwyp.js","../chunks/D-GAYjSq.js","../chunks/C8ToNXnz.js","../chunks/DA2DiQt1.js","../chunks/6Y7MQqFW.js","../chunks/BpeGWlSU.js","../assets/3.IuCLU1o4.css","../nodes/4.CYbwnt2w.js","../chunks/Dsah7X6o.js","../chunks/DON41-k1.js","../assets/Deckli

In [25]:
import subprocess, re

base = "https://j6e.me/mtg-meta-analyzer"

# Fetch the start JS to find route manifest
r = subprocess.run(['curl', '-sL', f"{base}/_app/immutable/entry/start.DfdyMplu.js"], capture_output=True, text=True, timeout=15)
start_js = r.stdout

# Look for route patterns, page references, component paths
routes = re.findall(r'"/[^"]*"', start_js)
print("Routes/paths found:")
for rt in sorted(set(routes)):
    print(f"  {rt}")

print(f"\nTotal start.js size: {len(start_js)} chars")

# Look for references to other JS chunks
chunks = re.findall(r'["\']([^"\']*\.js)["\']', start_js)
print(f"\nJS chunk references ({len(chunks)}):")
for c in sorted(set(chunks)):
    print(f"  {c}")

# Look for page/layout references
pages = re.findall(r'(?:page|layout|route|metagame|deck|match|chart|filter|tab)', start_js, re.I)
print(f"\nKeyword matches: {sorted(set(pages))}")

Routes/paths found:

Total start.js size: 118 chars

JS chunk references (2):
  ../chunks/C3NQ9sqC.js
  ../chunks/CaosO92D.js

Keyword matches: []


In [26]:
import subprocess, re

base = "https://j6e.me/mtg-meta-analyzer"

# All JS files referenced in the HTML
html_chunks = [
    "entry/start.DfdyMplu.js",
    "chunks/C3NQ9sqC.js",
    "chunks/C6U0rBQQ.js",
    "chunks/CaosO92D.js",
    "chunks/D-luxm3l.js",
    "entry/app.BvaApAJx.js",
    "chunks/B9QYiEl2.js",
    "chunks/Cqu7ILzR.js",
    "chunks/D-GAYjSq.js",
    "chunks/C6kBW4ne.js",
    "chunks/BpeGWlSU.js",
    "chunks/DON41-k1.js",
]

all_js = {}
for chunk in html_chunks:
    url = f"{base}/_app/immutable/{chunk}"
    r = subprocess.run(['curl', '-sL', url], capture_output=True, text=True, timeout=15)
    all_js[chunk] = r.stdout
    print(f"{chunk}: {len(r.stdout)} chars")

print(f"\nTotal JS fetched: {sum(len(v) for v in all_js.values())} chars")

entry/start.DfdyMplu.js: 118 chars
chunks/C3NQ9sqC.js: 20321 chars
chunks/C6U0rBQQ.js: 24029 chars
chunks/CaosO92D.js: 5744 chars
chunks/D-luxm3l.js: 372 chars
entry/app.BvaApAJx.js: 7174 chars
chunks/B9QYiEl2.js: 6697 chars
chunks/Cqu7ILzR.js: 1232 chars
chunks/D-GAYjSq.js: 1046 chars
chunks/C6kBW4ne.js: 1496 chars
chunks/BpeGWlSU.js: 302 chars
chunks/DON41-k1.js: 933 chars

Total JS fetched: 69464 chars


In [27]:
import re

# Search all JS for route manifest, page references, keywords
combined = '\n'.join(all_js.values())

# Find route/path patterns
print("=== ROUTE PATTERNS ===")
for m in re.finditer(r'(?:route|path|page|layout)[^;]{0,200}', combined, re.I):
    txt = m.group()
    if len(txt) > 10:
        print(f"  ...{txt[:150]}...")

print("\n=== METAGAME / META REFERENCES ===")
for m in re.finditer(r'[^\n]{0,80}(?:metagame|meta_|matchup|deck|archetype|format|chart|filter|tab)[^\n]{0,80}', combined, re.I):
    print(f"  {m.group()[:160]}")

print("\n=== IMPORT REFERENCES (additional modules) ===")
additional_js = set()
for m in re.finditer(r'import\s*\{[^}]*\}\s*from\s*["\']([^"\']+)["\']|import\s*["\']([^"\']+)["\']', combined):
    ref = m.group(1) or m.group(2)
    if ref and '.js' in ref:
        additional_js.add(ref)
for j in sorted(additional_js):
    print(f"  {j}")

# Also find dynamic imports
print("\n=== DYNAMIC IMPORTS ===")
for m in re.finditer(r'import\(["\']([^"\']+)["\']\)', combined):
    print(f"  {m.group(1)}")

=== ROUTE PATTERNS ===
  ...layouts:[0,...l||[]].map(o),leaf:s(c)}...
  ...layouts.length=Math.max(f.errors.length,f.layouts.length),f})...
  ...page:Le({}),navigating:Qe(null),updated:et()}...
  ...layouts,t.leaf].filter(Boolean).map(a=>a[1]()))}async function He(e,t,a){m=e.state...
  ...page),Be=new y.root({target:t,props:{...e.props,stores:I,components:Q},hydrate:a,sync:!1}),await Promise.resolve(),Ve(S),a){const r={from:null,to:{par...
  ...route:s,form:o}){let i="never"...
  ...pathname===x||e.pathname===x+"/"))i="always"...
  ...pathname=nt(e.pathname,i),e.search=e.search...
  ...route:s},props:{constructors:_t(a).map(f=>f.node.component),page:xe(R)}}...
  ...page={error:r,params:t,route:{id:s?.id??null},state:{},status:n,url:new URL(e),form:o??null,data:u?l:R.data}),c}async function Ee({loader:e,parent:t,u...
  ...route:!1,url:!1,search_params:new Set},c=await e()...
  ...route&&t||r.url&&a)return!0...
  ...route:a,params:n}){return{type:"loaded",state:{error:e,url:t,route:a,par

In [ ]:
import re

combined = '\n'.join(all_js.values())

# 1. Find dynamic imports (lazy-loaded pages)
print("=== DYNAMIC IMPORTS (lazy-loaded pages) ===")
for m in re.finditer(r'import\(["\']([^"\']+)["\']\)', combined):
    print(f"  {m.group(1)}")

# 2. Find route definitions  
print("\n=== ROUTE DEFINITIONS ===")
# SvelteKit stores routes in the manifest
for m in re.finditer(r'\[.*?(?:metagame|deck|match|format|card).*?\]', combined, re.I):
    print(f"  {m.group()[:200]}")

# 3. Find string literals that look like feature-related
print("\n=== FEATURE KEYWORDS in string literals ===")
keywords = set()
for m in re.finditer(r'["\']([^"\']{3,80})["\']', combined):
    val = m.group(1)
    lower = val.lower()
    if any(kw in lower for kw in ['metagame', 'matchup', 'deck', 'archetype', 'format', 'chart', 
                                     'filter', 'tab', 'table', 'population', 'win', 'rate',
                                     'color', 'mana', 'card', 'tournament', 'round', 'standing',
                                     'pie', 'bar', 'line', 'scatter', 'graph', 'heatmap',
                                     'select', 'dropdown', 'search', 'sort', 'export', 'download',
                                     'share', 'sidebar', 'nav', 'header', 'footer', 'modal',
                                     'tooltip', 'legend', 'axis', 'label']):
        keywords.add(val)

for kw in sorted(keywords)[:80]:
    print(f"  {kw}")

In [28]:
import subprocess, os
os.chdir('/workspaces/scrapermeleegg')

# 1. Run generate_site.py
print('=== Running generate_site.py ===')
result = subprocess.run(['/bin/python3', 'generate_site.py'], capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
print('STDOUT:')
print(result.stdout)
print('STDERR:')
print(result.stderr)
print('Return code:', result.returncode)

# 2. Check dist/ directory
print('\n=== ls -la dist/ ===')
result2 = subprocess.run(['ls', '-la', 'dist/'], capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
print(result2.stdout)
print(result2.stderr)

# 3. First 30 lines of dist/index.html
print('\n=== First 30 lines of dist/index.html ===')
try:
    with open('dist/index.html', 'r') as f:
        lines = f.readlines()[:30]
        for i, line in enumerate(lines, 1):
            print(f'{i:3d}: {line}', end='')
except FileNotFoundError:
    print('dist/index.html NOT FOUND')

=== Running generate_site.py ===
STDOUT:
[*] Cargando datos de: tournament_data.json
[*] Procesando metagame...
[*] Arquetipos con partidas: 104
[✓] Sitio generado en: dist/
    index.html  (724,415 bytes)
    _redirects

    Para Netlify: deploy la carpeta 'dist/'
    Para preview local: python -m http.server -d dist

STDERR:

Return code: 0

=== ls -la dist/ ===
total 724
drwxrwxrwx+ 2 codespace codespace   4096 Feb 23 17:29 .
drwxrwxrwx+ 7 codespace root        4096 Feb 23 17:29 ..
-rw-rw-rw-  1 codespace codespace     24 Feb 23 17:29 _redirects
-rw-rw-rw-  1 codespace codespace 726578 Feb 23 17:29 index.html



=== First 30 lines of dist/index.html ===
  1: <!DOCTYPE html>
  2: <html lang="es">
  3: <head>
  4: <meta charset="UTF-8">
  5: <meta name="viewport" content="width=device-width, initial-scale=1.0">
  6: <title>Meta Analyzer — Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am</title>
  7: <link rel="icon" href="data:image/svg+xml,<svg xmln

In [29]:
# Step 1: Just the return code and last 50 lines of output
import subprocess
result = subprocess.run(['/bin/python3', 'generate_site.py'], capture_output=True, text=True, cwd='/workspaces/scrapermeleegg')
stdout_lines = result.stdout.strip().split('\n') if result.stdout else []
stderr_lines = result.stderr.strip().split('\n') if result.stderr else []
print(f'Return code: {result.returncode}')
print(f'Total STDOUT lines: {len(stdout_lines)}')
print(f'Total STDERR lines: {len(stderr_lines)}')
print('\n--- Last 30 lines of STDOUT ---')
for line in stdout_lines[-30:]:
    print(line)
print('\n--- Last 30 lines of STDERR ---')
for line in stderr_lines[-30:]:
    print(line)

Return code: 0
Total STDOUT lines: 9
Total STDERR lines: 0

--- Last 30 lines of STDOUT ---
[*] Cargando datos de: tournament_data.json
[*] Procesando metagame...
[*] Arquetipos con partidas: 104
[✓] Sitio generado en: dist/
    index.html  (724,415 bytes)
    _redirects

    Para Netlify: deploy la carpeta 'dist/'
    Para preview local: python -m http.server -d dist

--- Last 30 lines of STDERR ---


In [30]:
# Step 2: ls -la dist/
import subprocess
result2 = subprocess.run(['ls', '-la', '/workspaces/scrapermeleegg/dist/'], capture_output=True, text=True)
print(result2.stdout)
print(result2.stderr)

total 724
drwxrwxrwx+ 2 codespace codespace   4096 Feb 23 17:29 .
drwxrwxrwx+ 7 codespace root        4096 Feb 23 17:29 ..
-rw-rw-rw-  1 codespace codespace     24 Feb 23 17:29 _redirects
-rw-rw-rw-  1 codespace codespace 726578 Feb 23 17:29 index.html




In [31]:
# Step 3: First 30 lines of dist/index.html
with open('/workspaces/scrapermeleegg/dist/index.html', 'r') as f:
    lines = f.readlines()[:30]
    for i, line in enumerate(lines, 1):
        print(f'{i:3d}: {line}', end='')

  1: <!DOCTYPE html>
  2: <html lang="es">
  3: <head>
  4: <meta charset="UTF-8">
  5: <meta name="viewport" content="width=device-width, initial-scale=1.0">
  6: <title>Meta Analyzer — Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am</title>
  7: <link rel="icon" href="data:image/svg+xml,<svg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 100 100'><text y='.9em' font-size='90'>🎴</text></svg>">
  8: <style>
  9: /* ═══════════════ RESET & BASE ═══════════════ */
 10: *,*::before,*::after{box-sizing:border-box;margin:0;padding:0}
 11: :root{
 12:   --bg-0:#0b0d14;--bg-1:#111320;--bg-2:#181b2c;--bg-3:#1f2340;
 13:   --border:#2a2f4a;--border-focus:#4f46e5;
 14:   --text-0:#f0f0f5;--text-1:#c5c7d6;--text-2:#8b8fa8;--text-3:#5c6080;
 15:   --accent:#6366f1;--accent-hover:#818cf8;
 16:   --green:#34d399;--yellow:#fbbf24;--red:#f87171;
 17:   --good:rgba(52,211,153,.12);--ok:rgba(251,191,36,.08);--bad:rgba(248,113,113,.12);
 18:   --radius:8px;--shadow:0 

In [2]:
# Levantar preview local del sitio
import subprocess, time

# Matar cualquier servidor previo en el puerto 8080
subprocess.run(['pkill', '-f', 'http.server.*8080'], capture_output=True)
time.sleep(0.5)

# Iniciar servidor HTTP en background
server = subprocess.Popen(
    ['python3', '-m', 'http.server', '-d', '/workspaces/scrapermeleegg/dist', '8080'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
time.sleep(1)

# Verificar que está corriendo
result = subprocess.run(['curl', '-s', '-o', '/dev/null', '-w', '%{http_code}', 'http://localhost:8080/'],
                       capture_output=True, text=True, timeout=5)
http_code = result.stdout.strip()

if http_code == '200':
    print(f'✅ Servidor corriendo en http://localhost:8080/')
    print(f'   PID: {server.pid}')
    print(f'   HTTP Status: {http_code}')
    
    # Verificar contenido
    result2 = subprocess.run(['curl', '-s', 'http://localhost:8080/'], capture_output=True, text=True, timeout=5)
    content_len = len(result2.stdout)
    has_title = 'Meta Analyzer' in result2.stdout
    print(f'   Content length: {content_len:,} bytes')
    print(f'   Contiene título: {has_title}')
    print(f'\n   📂 Abre en el navegador: http://localhost:8080/')
else:
    print(f'❌ Servidor no respondió correctamente (HTTP {http_code})')
    print(f'   stderr: {server.stderr.read().decode()[:300]}')

✅ Servidor corriendo en http://localhost:8080/
   PID: 16327
   HTTP Status: 200
   Content length: 724,415 bytes
   Contiene título: True

   📂 Abre en el navegador: http://localhost:8080/


In [3]:
# Commit y push a GitHub
import subprocess, os
os.chdir('/workspaces/scrapermeleegg')

# Configurar git si es necesario
subprocess.run(['git', 'config', 'user.name', 'torinescudo'], capture_output=True)
subprocess.run(['git', 'config', 'user.email', 'noreply@github.com'], capture_output=True)

# Agregar archivos relevantes (no dist/, no .venv/, no __pycache__/)
files_to_add = [
    '.gitignore',
    'README.md',
    'melee_scraper.py',
    'meta_analyzer.py',
    'generate_site.py',
    'netlify.toml',
    'matches.csv',
    'matchup_matrix.csv',
    'tournament_data.json',
    'melee_research.ipynb',
]

for f in files_to_add:
    result = subprocess.run(['git', 'add', f], capture_output=True, text=True)
    if result.returncode != 0:
        print(f'⚠️ Error al agregar {f}: {result.stderr.strip()}')

# Ver qué se va a commitear
status = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True)
print('=== Archivos staged ===')
print(status.stdout)

# Commit
commit = subprocess.run(
    ['git', 'commit', '-m', 'feat: scraper melee.gg + dashboard HTML + matchup matrix\n\n- melee_scraper.py: extrae matches vía DataTables endpoint\n- meta_analyzer.py: análisis con descarga de decklists\n- generate_site.py: genera dashboard estático autocontenido\n- 104 arquetipos, 5036 matches del Regional Championship SCG CON Milwaukee'],
    capture_output=True, text=True
)
print('\n=== Commit ===')
print(commit.stdout)
if commit.returncode != 0:
    print(f'stderr: {commit.stderr}')

# Push
push = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True, timeout=30)
print('\n=== Push ===')
print(push.stdout)
if push.returncode != 0:
    print(f'stderr: {push.stderr}')
else:
    print('✅ Push completado - Netlify debería iniciar el deploy automáticamente')

=== Archivos staged ===
A  .gitignore
M  README.md
A  generate_site.py
A  matches.csv
A  matchup_matrix.csv
A  melee_research.ipynb
A  melee_scraper.py
A  meta_analyzer.py
A  netlify.toml
A  tournament_data.json


=== Commit ===
[main 83a47cb] feat: scraper melee.gg + dashboard HTML + matchup matrix
 10 files changed, 83605 insertions(+), 2 deletions(-)
 create mode 100644 .gitignore
 create mode 100644 generate_site.py
 create mode 100644 matches.csv
 create mode 100644 matchup_matrix.csv
 create mode 100644 melee_research.ipynb
 create mode 100644 melee_scraper.py
 create mode 100644 meta_analyzer.py
 create mode 100644 netlify.toml
 create mode 100644 tournament_data.json


=== Push ===

✅ Push completado - Netlify debería iniciar el deploy automáticamente


In [1]:
# Validar el HTML generado
import os, json

dist_path = '/workspaces/scrapermeleegg/dist/index.html'
with open(dist_path, 'r', encoding='utf-8') as f:
    site_html = f.read()

print(f'Tamaño total: {len(site_html):,} bytes')
print(f'Líneas: {site_html.count(chr(10)):,}')

# Verificar componentes clave
checks = {
    'DOCTYPE html': '<!DOCTYPE html>' in site_html,
    'Título del torneo': 'Meta Analyzer' in site_html,
    'Nav tabs (3)': site_html.count('nav-btn') >= 3,
    'Panel metagame': 'meta-panel' in site_html,
    'Panel matchups': 'matrix-panel' in site_html,
    'Panel deck detail': 'detail-panel' in site_html,
    'Datos ARCHETYPES embebidos': 'const ARCHETYPES' in site_html,
    'Datos MX_DATA embebidos': 'const MX_DATA' in site_html,
    'Pie chart SVG': 'pie-svg' in site_html,
    'Matchup matrix table': 'mx-tbl' in site_html,
}

print('\n=== Validación de componentes ===')
all_ok = True
for label, ok in checks.items():
    status = '✅' if ok else '❌'
    if not ok: all_ok = False
    print(f'  {status} {label}')

# Extraer datos embebidos para verificar contenido
import re
m = re.search(r'const ARCHETYPES = (\[.*?\]);\s*const MX_DECKS', site_html, re.DOTALL)
if m:
    archetypes = json.loads(m.group(1))
    top5 = sorted(archetypes, key=lambda a: a['share'], reverse=True)[:5]
    print(f'\n=== Top 5 arquetipos embebidos ===')
    for i, a in enumerate(top5, 1):
        print(f'  {i}. {a["name"]}: {a["share"]}% share, {a["winrate"]}% WR ({a["total"]} matches)')
else:
    print('\n⚠️ No se pudieron extraer datos embebidos')

print(f'\n{"✅ Sitio validado correctamente" if all_ok else "❌ Hay componentes faltantes"}')

Tamaño total: 724,415 bytes
Líneas: 680

=== Validación de componentes ===
  ✅ DOCTYPE html
  ✅ Título del torneo
  ✅ Nav tabs (3)
  ✅ Panel metagame
  ✅ Panel matchups
  ✅ Panel deck detail
  ✅ Datos ARCHETYPES embebidos
  ✅ Datos MX_DATA embebidos
  ✅ Pie chart SVG
  ✅ Matchup matrix table

=== Top 5 arquetipos embebidos ===
  1. Izzet Lessons: 10.73% share, 53.06% WR (1159 matches)
  2. Mono-Green Landfall: 8.72% share, 54.68% WR (1026 matches)
  3. Dimir Excruciator: 8.47% share, 47.57% WR (824 matches)
  4. Dimir Midrange: 6.62% share, 50.73% WR (684 matches)
  5. Simic Rhythm: 6.45% share, 49.53% WR (644 matches)

✅ Sitio validado correctamente


# Validación y deploy

El sitio fue generado correctamente en `dist/index.html` (725 KB).
Pasos finales:
1. Validar estructura del HTML generado
2. Levantar preview local
3. Commit + push a GitHub para deploy en Netlify

In [4]:
# Importar el torneo existente a la nueva DB multi-torneo
import importlib, sys, os
sys.path.insert(0, '/workspaces/scrapermeleegg')
os.chdir('/workspaces/scrapermeleegg')

import manage_tournaments
importlib.reload(manage_tournaments)

db = manage_tournaments.TournamentDB()

# Importar el tournament_data.json existente
tid = db.import_from_json('tournament_data.json')
print(f'\nTorneos en DB: {len(db.list_tournaments())}')
for t in db.list_tournaments():
    print(f'  [{t["id"]}] {t["name"]} — {t["total_matches"]} matches')

[✓] Importado: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (ID: 339227)

Torneos en DB: 1
  [339227] Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am — 5046 matches


In [6]:
# Regenerar sitio multi-torneo
import generate_site
importlib.reload(generate_site)

generate_site.generate_multi_tournament_site(db.data['tournaments'], 'dist')

[*] Procesando: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    Arquetipos con partidas: 104

[✓] Sitio generado en: dist/
    index.html  (727,106 bytes)
    Torneos embebidos: 1
      • Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (5046 matches)


In [7]:
# ═══ PEGA AQUÍ LA URL DEL TORNEO ═══
TOURNAMENT_URL = "https://melee.gg/Tournament/View/339227"  # <-- cambia esta URL

# ────────────────────────────────────
import importlib, manage_tournaments, generate_site
importlib.reload(manage_tournaments)
importlib.reload(generate_site)

db = manage_tournaments.TournamentDB()
tid = manage_tournaments.extract_tournament_id(TOURNAMENT_URL)

if tid in db.data['tournaments']:
    print(f"✅ Torneo {tid} ya existe en la base de datos — cargado.")
    t = db.data['tournaments'][tid]
    print(f"   📋 {t['tournament']['name']}")
    print(f"   📊 {t['tournament']['total_matches']} partidas, {len(t['matches'])} registros")
else:
    print(f"🔍 Torneo {tid} no encontrado en DB — scrapeando melee.gg...")
    db.add_tournament(TOURNAMENT_URL)
    print(f"✅ Torneo añadido correctamente.")

# Regenerar sitio con todos los torneos
print(f"\n🔄 Regenerando sitio con {len(db.data['tournaments'])} torneo(s)...")
generate_site.generate_multi_tournament_site(db.data['tournaments'], 'dist')
print("🎉 ¡Listo! Abre dist/index.html o despliega a Netlify.")

✅ Torneo 339227 ya existe en la base de datos — cargado.
   📋 Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
   📊 5046 partidas, 5036 registros

🔄 Regenerando sitio con 1 torneo(s)...
[*] Procesando: Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am
    Arquetipos con partidas: 104

[✓] Sitio generado en: dist/
    index.html  (727,106 bytes)
    Torneos embebidos: 1
      • Regional Championship - SCG CON Milwaukee - Season 4 - Round 2 - Saturday - 9:00 am (5046 matches)
🎉 ¡Listo! Abre dist/index.html o despliega a Netlify.


In [ ]:
# Git commit & push multi-tournament feature
import subprocess

# Stage files
subprocess.run(['git', 'add', 'manage_tournaments.py', 'tournaments_db.json', 'generate_site.py', 'melee_research.ipynb'], 
               capture_output=True, text=True)

# Commit
commit = subprocess.run(
    ['git', 'commit', '-m', 'feat: multi-tournament support with DB and selector UI\n\n- Add manage_tournaments.py: CLI/programmatic tournament DB manager\n- Refactor generate_site.py for multi-tournament HTML with selector dropdown\n- Add tournaments_db.json persistent database\n- Add interactive notebook cell for pasting tournament URLs'],
    capture_output=True, text=True
)
print(commit.stdout or commit.stderr)

# Push
push = subprocess.run(['git', 'push'], capture_output=True, text=True)
print(push.stdout or push.stderr)

## ➕ Añadir torneo nuevo

Pega la URL de melee.gg (ej: `https://melee.gg/Tournament/View/123456`) en la variable `TOURNAMENT_URL` y ejecuta la celda. Si ya está scrapeado se cargará de la base de datos; si no, se scrapeará automáticamente.

# Multi-torneo: importar torneo existente y regenerar

Nuevo sistema: `manage_tournaments.py` gestiona una DB de torneos.
`generate_site.py` genera el sitio con TODOS los torneos y un selector.